In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [3]:
## HACK get rid of it
import sys
sys.path.append("D:/ProgProjekte/Python/Re-stained_WSIs_Registration/src")

from wsi_registration import TissueDetector, MatcherParameters, WSI_Matcher

In [4]:
source_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\Aperio\CCMCT").glob("*.svs"))

if len(source_slide_paths) == 0:
    source_slide_paths = list(Path(r"/data/ScannerStudy/Aperio/CCMCT").glob("*.svs"))

len(source_slide_paths)

5

In [5]:
target_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\AxioScan\CCMCT").glob("*.tif")) 


if len(target_slide_paths) == 0:
    target_slide_paths = list(Path(r"/data/ScannerStudy/AxioScan/CCMCT").glob("*.tif"))

len(target_slide_paths)

5

In [6]:
source_slide_paths = {name.stem.split("_")[-2] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_paths.keys()

dict_keys(['183715A', '22108', '29609B', '380609B', '518711B'])

In [7]:
target_slide_paths = {name.stem.split("_")[-2] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_paths.keys()

dict_keys(['183715A', '22108', '29609B', '380609B', '518711B'])

In [8]:
for name in source_slide_paths:
    if name not in target_slide_paths:
        print(name)

In [9]:
for name in target_slide_paths:
    if name not in source_slide_paths:
        print(name)

In [25]:
%%time

for name, source in source_slide_paths.items():
    
    target = target_slide_paths[name]
    
    fixed_wsi = source["slide"]._filename
    float_wsi = target["slide"]._filename
    
    print(f"Source: {fixed_wsi} \n Target: {float_wsi}")
    
    for i in range(5):
        tissue_detector = TissueDetector("LAB_Threshold", threshold=80)

        matcher_parameters = MatcherParameters() 
        matcher = WSI_Matcher(tissue_detector, matcher_parameters)

        offset = matcher.match(fixed_wsi, float_wsi)

        print(f"{offset}")
    
    break
    

Source: D:\Datasets\ScannerStudy\Aperio\CCMCT\A_CCMCT_183715A_1.svs 
 Target: D:\Datasets\ScannerStudy\AxioScan\CCMCT\Z_CCMCT_183715A_1.tif
(-2080.47445740913, -2362.619890345375)
(-5463.47445740913, -1682.619890345375)
(2426.52554259087, -2161.619890345375)
(1153.52554259087, -3542.619890345375)
(-2925.47445740913, -1725.619890345375)
Wall time: 3min 29s
